In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pbspark import MessageConverter
from canbus_test_pb2 import CANBusMessage
import cantools

In [2]:
jarsPackages = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,com.datastax.spark:spark-cassandra-connector_2.12:3.2.0"

In [3]:
spark = SparkSession.builder.master("local[*]") \
                    .appName('Spark Structured Streaming from Sensor EV-BUS') \
                    .config("spark.jars.packages", jarsPackages) \
                    .getOrCreate()

23/05/17 18:45:00 WARN Utils: Your hostname, big-data-infra resolves to a loopback address: 127.0.1.1; using 10.252.62.71 instead (on interface ens18)
23/05/17 18:45:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/root/.local/share/virtualenvs/spark-processor-pBWFVQAw/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ffbeacc3-0c8e-451b-970a-3d60880a845b;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.1 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in centr

23/05/17 18:45:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/17 18:45:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
kafkaStream = spark.readStream \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "10.252.62.70:9092") \
      .option("subscribe", 'canbus_test') \
      .option("includeHeaders", "true") \
      .option('startingOffsets', 'latest') \
      .load() \
      .selectExpr("value", "headers",'timestamp')

In [5]:
dbc = cantools.database.load_file('./j1939.dbc')
dbc.add_dbc_file('iso.dbc')

Overwriting message 'LC' with 'LCMD' in the frame id to message dictionary because they have identical masked frame ids 0xcfe41fe.
Overwriting message 'GPV4' with 'GPV4' in the name to message dictionary.
Overwriting message 'GPV4' with 'GPV4' in the frame id to message dictionary because they have identical masked frame ids 0x1807fefe.
Overwriting message 'ECUID' with 'ECUID' in the name to message dictionary.
Overwriting message 'ECUID' with 'ECUID' in the frame id to message dictionary because they have identical masked frame ids 0x18fdc5fe.
Overwriting message 'LD' with 'LD' in the name to message dictionary.
Overwriting message 'LD' with 'LD' in the frame id to message dictionary because they have identical masked frame ids 0x18fe40fe.
Overwriting message 'SOFT' with 'SOFT' in the name to message dictionary.
Overwriting message 'SOFT' with 'SOFT' in the frame id to message dictionary because they have identical masked frame ids 0x18fedafe.
Overwriting message 'TD' with 'TD' in the

In [6]:
def convertToFloat(dictData):
        return dict([key, float(value)]
            for key, value in dictData.items())

In [7]:
# def writeSensorCanCassandra(batch_df,batch_id):
#     return batch_df.write \
#         .format("org.apache.spark.sql.cassandra") \
#         .options(table="sensor_data_bus", keyspace="busev") \
#         .mode('append') \
#         .save()

In [8]:
mc = MessageConverter()

In [9]:
def parsedRawData(df, proto):
    # Parsed Protobuf encryption
    parsedData = df.withColumn('parsed', mc.from_protobuf('value', proto)) \
        .withColumn('topic', expr('headers')[0]['value'].cast('string')) \
        .withColumn('bus_id', expr('headers')[2]['value'].cast('string')) \
        .select('topic','bus_id', 'parsed.*') \
        .withColumn('timestamp', to_timestamp(col('timestamp') / 1000))
    
    return parsedData

In [10]:
LONG_MESSAGE = {}

def parser(ID_HEX,DLC,DATA_HEX_STR):
        ID_HEX = int.from_bytes(ID_HEX, "big")
        DATA_LEN = int(DLC)
        PRIORITY = ID_HEX & (0b00011100 << 24)
        RESERVED = ID_HEX & (0b00000010 << 24)
        DATA_PAGE = ID_HEX & (0b00000001 << 24)
        PDU_FORMAT = ID_HEX & (0b11111111 << 16)
        PDU_SPECIFIC = ID_HEX & (0b11111111 << 8)
        SOURCE_ADDRESS = ID_HEX & (0b11111111 << 0)
        
        PGN = RESERVED | DATA_PAGE | PDU_FORMAT | PDU_SPECIFIC
        DBC_ID = PRIORITY | PGN | 0xFE
        try:
            currMsg = dbc.get_message_by_frame_id(DBC_ID)
            try:
                outdata = dbc.decode_message(DBC_ID,DATA_HEX_STR,decode_choices=True)
                return str(outdata)
            except:
                if currMsg.frame_id in LONG_MESSAGE.keys():
                    LONG_MESSAGE[currMsg.frame_id] = f"{LONG_MESSAGE[currMsg.frame_id]}{DATA_HEX_STR}"
                    try:
                        outdata = dbc.decode_message(DBC_ID,LONG_MESSAGE[currMsg.frame_id],decode_choices=True)
                        del LONG_MESSAGE[currMsg.frame_id]
                        return str(outdata)
                    except:
                        pass
                else:
                    LONG_MESSAGE[currMsg.frame_id] = DATA_HEX_STR
        except:
            pass

In [11]:
def parseMessageName(ID_HEX):
    ID_HEX = int.from_bytes(ID_HEX, "big")
    PRIORITY = ID_HEX & (0b00011100 << 24)
    RESERVED = ID_HEX & (0b00000010 << 24)
    DATA_PAGE = ID_HEX & (0b00000001 << 24)
    PDU_FORMAT = ID_HEX & (0b11111111 << 16)
    PDU_SPECIFIC = ID_HEX & (0b11111111 << 8)
    
    PGN = RESERVED | DATA_PAGE | PDU_FORMAT | PDU_SPECIFIC
    DBC_ID = PRIORITY | PGN | 0xFE
    print(DBC_ID)
    try:
        currMsg = dbc.get_message_by_frame_id(DBC_ID)
        try: 
            return currMsg.name
        except:
               pass
    except:
        pass

In [12]:
parsedCan = parsedRawData(kafkaStream,CANBusMessage)
parsedCan.printSchema()

root
 |-- topic: string (nullable = true)
 |-- bus_id: string (nullable = true)
 |-- canId: binary (nullable = true)
 |-- dlc: integer (nullable = true)
 |-- data: binary (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [13]:
asd = parsedCan.withColumn('newvalue', mc.from_protobuf('canId', CANBusMessage)) \
        .select('topic','bus_id','newvalue.*')
asd.printSchema()

root
 |-- topic: string (nullable = true)
 |-- bus_id: string (nullable = true)
 |-- canId: binary (nullable = true)
 |-- dlc: integer (nullable = true)
 |-- data: binary (nullable = true)
 |-- timestamp: long (nullable = true)



In [14]:
parserMessage = udf(lambda m,n,o: parser(m,n,o),MapType(StringType(),StringType()))

In [15]:
parserMessageName = udf(lambda m: parseMessageName(m))

In [16]:
parser = asd.withColumn("value", parserMessage('canId','dlc','data')) \
        .withColumn('message_name',when(parserMessageName('canId').isNull(),'null')
                    .otherwise(parserMessageName('canId'))) \
        .withColumnRenamed('canId','can_id')
        
parser.printSchema()

root
 |-- topic: string (nullable = true)
 |-- bus_id: string (nullable = true)
 |-- can_id: binary (nullable = true)
 |-- dlc: integer (nullable = true)
 |-- data: binary (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- value: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- message_name: string (nullable = true)



In [17]:
def writeToConsole(ds,OutputMode):
    return ds.writeStream \
            .outputMode(OutputMode) \
            .format('console') \
            .option('Vertical',True) \
            .option('Truncate',False) \
            .start()
        

In [ ]:
query = writeToConsole(parser,'append').awaitTermination()

23/05/17 18:45:11 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-76499f73-b5cf-4260-ab81-c2b9d8c48dcd. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/05/17 18:45:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
-------------------------------------------
Batch: 0
-------------------------------------------
+-----+------+------+---+----+---------+-----+------------+
|topic|bus_id|can_id|dlc|data|timestamp|value|message_name|
+-----+------+------+---+----+---------+-----+------------+
+-----+------+------+---+----+---------+-----+------------+



23/05/17 18:45:19 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/05/17 18:45:19 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/05/17 18:45:19 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/05/17 18:45:19 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/05/17 18:45:19 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
23/05/17 18:45:19 WARN KafkaDataConsumer: KafkaDataConsumer is not running in Un

418581758
419348478
418383358
418382334
217769726
418382334
217055998
418382334
418382334
418382334


-------------------------------------------
Batch: 1
-------------------------------------------
+-----------+------+-------------+---+-------------------------+-------------+-----+------------+
|topic      |bus_id|can_id       |dlc|data                     |timestamp    |value|message_name|
+-----------+------+-------------+---+-------------------------+-------------+-----+------------+
|canbus_test|bus_37|[18 F3 0C 00]|8  |[F3 3F 0F FF F0 FF CC 81]|1684318412055|null |null        |
|canbus_test|bus_35|[18 FE BF 0B]|8  |[00 00 7D 7D 7D 7D FF FF]|1684318632419|null |EBC2        |
|canbus_test|bus_34|[18 F0 05 03]|8  |[FF FF FF 7D FF FF FF FF]|1684317762243|null |ETC2        |
|canbus_test|bus_33|[18 F0 01 0B]|8  |[00 00 F3 FF FF 01 FF F7]|1684317149933|null |EBC1        |
|canbus_test|bus_36|[0C FA E6 21]|8  |[00 10 00 00 0C 01 00 63]|1684317934559|null |null        |
|canbus_test|bus_37|[18 F0 01 0B]|8  |[00 00 F3 FF FF 01 FF F7]|1684318413070|null |EBC1        |
|canbus_test|bus_35|[

419351294>                                                          (0 + 1) / 1]
217055998
419348478
418382334
419351294
419348478
419348478
419351294
419173630
217055998
419348478
418382334
419348478
419348478
418382334
419320318
419348478
419348478
418382334
418382334


...

4193484784:>                                                        (0 + 1) / 1]
418382334
419343870
418382334
418579710


-------------------------------------------
Batch: 244
-------------------------------------------
+-----------+------+-------------+---+-------------------------+-------------+-----+------------+
|topic      |bus_id|can_id       |dlc|data                     |timestamp    |value|message_name|
+-----------+------+-------------+---+-------------------------+-------------+-----+------------+
|canbus_test|bus_37|[18 FE BF 0B]|8  |[00 00 7D 7D 7D 7D FF FF]|1684318672108|null |EBC2        |
|canbus_test|bus_34|[18 F0 01 0B]|8  |[00 00 F3 FF FF 01 FF F7]|1684318023475|null |EBC1        |
|canbus_test|bus_35|[18 FE AD 0B]|8  |[00 00 00 00 FE FE FF FF]|1684318894114|null |EBC3        |
|canbus_test|bus_36|[18 F0 01 0B]|8  |[00 00 F3 FF FF 01 FF F7]|1684318196360|null |EBC1        |
|canbus_test|bus_33|[18 F3 04 25]|8  |[F3 FF AA EA FF FF 0A 00]|1684317411746|null |null        |
+-----------+------+-------------+---+-------------------------+-------------+-----+------------+

23/05/17 18:49:28 

4183820785:>                                                        (0 + 1) / 1]
419348478
418583806
419348478
217056510


-------------------------------------------
Batch: 245
-------------------------------------------
+-----------+------+-------------+---+-------------------------+-------------+-----+------------+
|topic      |bus_id|can_id       |dlc|data                     |timestamp    |value|message_name|
+-----------+------+-------------+---+-------------------------+-------------+-----+------------+
|canbus_test|bus_37|[18 F0 00 10]|8  |[FF 7D FF FF FF FF FF 7D]|1684318673118|null |ERC1        |
|canbus_test|bus_34|[18 FE BF 0B]|8  |[00 00 7D 7D 7D 7D FF FF]|1684318024501|null |EBC2        |
|canbus_test|bus_35|[18 F3 14 00]|8  |[69 00 C7 0C C7 0C C8 0C]|1684318895154|null |null        |
|canbus_test|bus_36|[18 FE BF 0B]|8  |[00 00 7D 7D 7D 7D FF FF]|1684318197370|null |EBC2        |
|canbus_test|bus_33|[0C F0 04 00]|8  |[FF FF FF 00 00 FF FF FF]|1684317412738|null |EEC1        |
+-----------+------+-------------+---+-------------------------+-------------+-----+------------+

23/05/17 18:49:29 

1508922866:>                                                        (0 + 1) / 1]
215105790
150892286
419351294
419351294
